In [16]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset
from copy import deepcopy
from transformers import (AutoTokenizer,
                         BertForSequenceClassification,
                         Trainer,
                         TrainingArguments,
                         TrainerCallback,
                         get_linear_schedule_with_warmup
                          )
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
# import wandb
# wandb.login()
# wandb.init(project="synthetic_tweets")
import ast
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import os
print(os.getpid())
import re

3965


In [17]:
df_synthetic = pd.read_json("/mntssd/mnt3/shanshanbai/nlpinearthobservation/synthetic_data/6000/merged_data.jsonl", lines=True)
df_synthetic['tweets_concatenated'] = df_synthetic['tweets'].apply(lambda x: ' '.join([item for item in x if item]) if isinstance(x, list) else '')
df_synthetic['mapped_class'] = ['residential'] * 3000 + ['commercial'] * (len(df_synthetic) - 3000)
df_synthetic.sort_values(by='building_id', inplace=True)
df_synthetic.head()

,building_id,building_city,building_tags,building_name,tweet_language_distribution,tweets,tweets_concatenated,mapped_class
0,31061,CapeTown,dormitory,Kopano,"[English, English, English, English, English]",[Midterm exams are looming and I'm stuck here ...,Midterm exams are looming and I'm stuck here h...,residential
1,31073,CapeTown,dormitory,Baxter Hall,"[English, English, English, English, English]",[Cramming for exams is way more bearable when ...,Cramming for exams is way more bearable when y...,residential
2,31079,CapeTown,dormitory,Graça Machel Hall,"[English, English, English, English, Polish]",[Morning views from Graça Mached don't get muc...,Morning views from Graça Mached don't get much...,residential
3,57201,Berlin,apartments,Doubletree By Hilton Berlin Ku'Damm,[German],[Eine Woche im Doubletree und ich fühle mich w...,Eine Woche im Doubletree und ich fühle mich wi...,residential
3000,103615,Munich,office,Wacker Chemie Ag,"[German, German, German, German, English]",[Bei so einem Wetter wie heute freue ich mich ...,Bei so einem Wetter wie heute freue ich mich b...,commercial


In [18]:
df_real = pd.read_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/real.csv')
df_real = df_real[df_real['building_id'].isin(df_synthetic['building_id'])]
df_real['tweet_no_url'] = df_real['tweet_no_url'].apply(ast.literal_eval)
df_real['tweets_concatenated'] = df_real['tweet_no_url'].apply(lambda x: ' '.join([item for item in x if item]) if isinstance(x, list) else '')
df_real.sort_values(by='building_id', inplace=True)
df_real.head()

,building_id,mapped_class,language,city,tweet_no_url,tweets_concatenated
0,31061,residential,"['en', 'en', 'en', 'en', 'in', 'en', 'en', 'lt...",CapeTown,[Saturday mornings will never be the same agai...,Saturday mornings will never be the same again...
1,31073,residential,"['en', 'en', 'en', 'en', 'en']",CapeTown,"[Just posted a photo @ Baxter Hall , Happiness...",Just posted a photo @ Baxter Hall Happiness i...
2,31079,residential,"['en', 'en', 'pl', 'en', 'en']",CapeTown,"[Feels good to be back. 😭🙏, Students, did you ...","Feels good to be back. 😭🙏 Students, did you mi..."
3,57201,residential,['de'],Berlin,[Kunstgalerie Robert Grunenberg ],Kunstgalerie Robert Grunenberg
4,103615,commercial,"['de', 'en', 'de', 'de']",Munich,"[I'm at Neuperlach in München , Bradley Co Sch...",I'm at Neuperlach in München Bradley Co Schoo...


In [19]:
# df_real = pd.read_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/Data_Train_Test/first_paper/concatenated_imbalance_nobrokenimage_train_multilingual.csv')
# df_real = df_real[~df_real['building_id'].isin(df_synthetic['building_id'])]
# residential_sample = df_real[df_real['mapped_class'] == 'residential'].sample(n=1000, random_state=42)
# commercial_sample = df_real[df_real['mapped_class'] == 'commercial'].sample(n=1000, random_state=42)

# # Combine both samples into one DataFrame
# df_real = pd.concat([residential_sample, commercial_sample])

# # Function to remove URLs
# def remove_urls(text):
#     url_pattern = r'http[s]?://\S+|www\.\S+'  # Matches URLs
#     return re.sub(url_pattern, '', text)

# # Apply function to the column
# df_real['concatenated_original'] = df_real['concatenated_original'].apply(remove_urls)



In [20]:
# # # train on synthetic and test on real

# # # First, split into train + eval and test
# # X_synthetic = df_synthetic['tweets_concatenated'].values.tolist()
# # X_real = df_real['tweets_concatenated'].values.tolist()
# # y_synthetic = df_synthetic['mapped_class'].values.tolist()
# # y_real = df_real['mapped_class'].values.tolist()
# # # First, split into train + eval and test
# # X_train_eval, X_test, y_train_eval, y_test = train_test_split(X_synthetic, y_synthetic, test_size=0.2, random_state=5)
# # # Then, split train + eval into train and eval
# # X_train, _, y_train, _= train_test_split(X_train_eval, y_train_eval, test_size=0.2, random_state=5)  # 0.25 * 0.8 = 0.2

# # # _, X_test, _, y_test = train_test_split(X_real, y_real, test_size=0.2, random_state=4)


# # # print(len(X_train))
# # # print(len(X_eval))
# # # print(len(X_test))


# X_train = df_synthetic['tweets_concatenated'].values.tolist()
# X_test = df_real['tweets_concatenated'].values.tolist()
# y_train = df_synthetic['mapped_class'].values.tolist()
# y_test = df_real['mapped_class'].values.tolist()

# X_train_eval, _, y_train_eval, _ = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
# X_train, _, y_train, _= train_test_split(X_train_eval, y_train_eval, test_size=0.2, random_state=0)  # 0.25 * 0.8 = 0.2
# # X_test = df_real['concatenated_original'].values.tolist()
# # y_test = df_real['mapped_class'].values.tolist()


In [21]:
# First, split into train + eval and test
X_synthetic = df_synthetic['tweets_concatenated'].values.tolist()
X_real = df_real['tweets_concatenated'].values.tolist()
y_synthetic = df_synthetic['mapped_class'].values.tolist()
y_real = df_real['mapped_class'].values.tolist()
# First, split into train + eval and test
X_train_eval, _, y_train_eval, _ = train_test_split(X_synthetic, y_synthetic, test_size=0.2, random_state=0)

X_train, X_eval, y_train, y_eval = train_test_split(X_train_eval, y_train_eval, test_size=0.2, random_state=0)
_, X_test, _, y_test = train_test_split(X_real, y_real, test_size=0.2, random_state=0)



# # Convert text to numerical features
# vectorizer = TfidfVectorizer()
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# # Train Naive Bayes
# model = MultinomialNB()
# model.fit(X_train_tfidf, y_train)

# # Predict and evaluate
# y_pred = model.predict(X_test_tfidf)
# print(classification_report(y_test, y_pred))

In [22]:
CHECK_POINT = "google-bert/bert-base-multilingual-cased"
CACHE_DIR = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/.cache/huggingface/hub"  # Specify your custom directory

# Load tokenizer and model with the specified cache directory
tokenizer = AutoTokenizer.from_pretrained(CHECK_POINT)
model = BertForSequenceClassification.from_pretrained(CHECK_POINT, cache_dir=CACHE_DIR, num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
def encoding_and_labels(X, y):
    sentences = X
    label_str = y
    labels = [0 if label == "residential" else 1 for label in label_str]
    encodings = tokenizer(sentences, truncation=True, padding=True, return_tensors="pt") # , is_split_into_words=True
    return encodings, labels

train_encodings, train_labels = encoding_and_labels(X_train, y_train)
eval_encodings, eval_labels = encoding_and_labels(X_eval, y_eval)
test_encodings, test_labels = encoding_and_labels(X_test, y_test)

In [24]:
class TwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = TwitterDataset(train_encodings, train_labels)
eval_dataset = TwitterDataset(eval_encodings, eval_labels)
test_dataset = TwitterDataset(test_encodings, test_labels)


In [25]:
def compute_metrics(eval_prediction):
    predictions = eval_prediction.predictions.argmax(-1)
    labels = eval_prediction.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    metrics = {'accuracy': acc,
               'precision': precision,
               'recall': recall,
               'f1': f1}
    return metrics

In [26]:
OUTPUT_DIR = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/output_synthetic"
LOGGING_DIR ="/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/output_synthetic"
NUM_EPOCHS = 5
TRAIN_BATCH_SIZE = 32 #8
EVAL_BATCH_SIZE = 32 #32
WARMUP_RATIO = 0.01 # 200
WEIGHT_DECAY = 0.01
LOGGING_STEPS = 100 # 1000
LEARNING_RATE = 5e-6

# Calculate total training steps
total_train_steps = (len(train_labels) // TRAIN_BATCH_SIZE) * NUM_EPOCHS
print(total_train_steps)

# Calculate the saving steps to be aligned with logging steps
SAVING_STEPS = LOGGING_STEPS

600


In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    warmup_ratio= WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
    logging_dir=LOGGING_DIR,
    evaluation_strategy="steps",
    logging_steps=LOGGING_STEPS,
    save_steps=SAVING_STEPS,  # Set saving steps to align with logging steps
    learning_rate=LEARNING_RATE,
    metric_for_best_model="f1"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  #{'train':train_dataset,'eval':eval_dataset},
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shanshan-bai (nlp-in-earth-observation). Use `wandb login --relogin` to force relogin


/tmp/ipykernel_3965/1560154188.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.475100,0.188659,0.941667,0.941667,0.941858,0.941660
200,0.146600,0.100482,0.975000,0.974965,0.975163,0.974996
300,0.110300,0.085762,0.976042,0.976376,0.975836,0.976020
400,0.065000,0.081535,0.980208,0.980442,0.980048,0.980193


/tmp/ipykernel_3965/1560154188.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_3965/1560154188.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_3965/1560154188.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
# model_trained = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/output_synthetic/checkpoint-100"  # Specify your custom directory

# model = BertForSequenceClassification.from_pretrained(model_trained, num_labels=2)

In [ ]:
def compute_metrics(eval_prediction):
    predictions = eval_prediction.predictions.argmax(-1)
    labels = eval_prediction.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    metrics = {'accuracy': acc,
               'precision': precision,
               'recall': recall,
               'f1': f1}
    
    # convert label_id to label name
    mapping = {0: "residential", 1: "commercial"} 
    labels = [mapping[label_id] for label_id in labels]
    predictions = [mapping[prediction] for prediction in predictions]
    
    # create classification report
    report = classification_report(labels, predictions, zero_division=0)
    print("Classification Report:\n\n\n", report)
    
    return metrics

In [ ]:
import warnings
warnings.filterwarnings('ignore')

test_args = TrainingArguments(
    output_dir='/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/output_synthetic',
    do_train=False,
    do_predict=True,
    per_device_eval_batch_size=32,   
    dataloader_drop_last=False,
)


trainer = Trainer(
    model=model,
    args=test_args,
    compute_metrics=compute_metrics,
) 

test_results = trainer.predict(test_dataset)

In [ ]:
# # Extract predicted class indices
# y_pred = np.argmax(test_results.predictions, axis=1)
# # Extract true labels from test_dataset (assuming test_dataset has labels)
# y_test = test_dataset.labels # Adjust if labels are stored differently

# # Find indices where predictions do not match actual labels
# misclassified_indices = np.where(y_test != y_pred)[0]

# # Create a DataFrame of misclassified samples
# misclassified_samples = pd.DataFrame({
#     'Text': [X_test[i] for i in misclassified_indices],
#     'Actual Label': [y_test[i] for i in misclassified_indices],
#     'Predicted Label': [y_pred[i] for i in misclassified_indices]
# })
# misclassified_samples